# Requirements
- A project must exist in [the Firebase Console](https://console.firebase.google.com/?_gl=1*rk62ap*_ga*NDcxNDU2ODM2LjE2OTUzNTg5NjE.*_ga_CW55HF8NVT*MTY5NTg3NDI1Ni44LjEuMTY5NTg3ODU0Ny42MC4wLjA).
- The project needs to have Authentication turned on and configure, and the email and password provider must be enabled.
- [Google Cloud CLI](https://cloud.google.com/sdk/docs/install?hl=pt-br) must be installed at least on the configuring machine.

# Enabling MFA on a project level
This only needs to run once, it can be run as a REST request or as method calls in Firebase Admin SDK

In [ ]:
$PROJECT_ID = "⚠️Sensitive data⚠️"
$gcp = gcloud auth print-access-token
$curlUrl = "https://identitytoolkit.googleapis.com/admin/v2/projects/"+$PROJECT_ID+"/config?updateMask=mfa"
$curlAuthHeader = "Authorization: Bearer "+$gcp
#adjacentIntervals: The number of time-window intervals, from zero to ten. The default is five.
$curlBody = @"
{ \"mfa\": { \"providerConfigs\": [{  \"state\": \"ENABLED\",  \"totpProviderConfig\": {    \"adjacentIntervals\": \"5\"  }}] }}
"@

curl -X PATCH $curlUrl -H  $curlAuthHeader -H "Content-Type: application/json" -H "X-Goog-User-Project: "+$PROJECT_ID -d $curlBody

# Registration
After MFA is enabled at a project level, the user must be registred in firebase and have its email address verified before enrolling.
This can be done through REST requests or Firebase Auth Javascript SDK. 
This sample focuses on C# and .NET applications, so it uses [an open source package](https://github.com/step-up-labs/firebase-authentication-dotnet) to simplify some flows, specifically user creation and login, but that also could be implemented on different ways.

In [ ]:
#r "nuget: FirebaseAuthentication.net"
#r "nuget: System.Net.Http.Json"

using System.Net.Http.Json;
using Firebase.Auth;
using Firebase.Auth.Providers;

var config = new FirebaseAuthConfig
{
    ApiKey = "⚠️Sensitive data⚠️",
    AuthDomain = "⚠️Sensitive data⚠️",
    Providers = new FirebaseAuthProvider[]
    {
        new EmailProvider()
    }
};

var client = new FirebaseAuthClient(config);

Now that the package is configured, we will use it to create a new firebase user.

In [ ]:
UserCredential userCredential;
var email = "⚠️Sensitive data⚠️";
Console.WriteLine($"Using email: {email}");
try
{
    var firebasePassword = "⚠️Sensitive data⚠️";
    var displayName = "Firebase TOTP Test User";
    userCredential = await client.CreateUserWithEmailAndPasswordAsync(email, firebasePassword, displayName);
}
catch (FirebaseAuthException ex)
{
    Console.WriteLine($"Exception thrown: {ex.Reason}");
    throw;
}

Console.WriteLine($"Logged in as: {userCredential.User.Uid} | {userCredential.User.Info.DisplayName} | {userCredential.User.Info.Email}");

Now for the email verification.

In [ ]:
var token = await userCredential.User.GetIdTokenAsync(true);

var sendVerifyEmailRequest = new {
    requestType = "VERIFY_EMAIL",
    idToken = token
};

var sendVerifyEmailResponse = await config.HttpClient.PostAsJsonAsync($"https://identitytoolkit.googleapis.com/v1/accounts:sendOobCode?key={config.ApiKey}", sendVerifyEmailRequest);

## ⚠️Please be sure to verify the email address before continuing.
# Enrollment
Using the user's credentials, we can start the MFA TOTP enrollment process.
An QR Code needs to be show to the user and instructions should be made clear on how to use OTP apps. OTP code must also be avaiable because some plataforms dont support QR Code.

In [ ]:
public class TotpSessionInfoWrapper
{
    public Content TotpSessionInfo { get; set; }

    public class Content
    {
        public string SharedSecretKey { get; set; }
        public string HashingAlgorithm { get; set; }
        public int VerificationCodeLength { get; set; }
        public int PeriodSec { get; set; }
        public string SessionInfo { get; set; }
        public DateTime FinalizeEnrollmentTime { get; set; }
    
        public string GetQRCodeUrl(string accountName = default, string issuer = default)
        {
            return $"otpauth://totp/{issuer}:{accountName}?secret={SharedSecretKey}&issuer={issuer}&algorithm={HashingAlgorithm}&digits={VerificationCodeLength}";
        }
    }
}

var startEnrollmentRequest = new
{
    idToken = token,
    totpEnrollmentInfo = new object()
};

var startEnrollmentResponse = await client.PostAsJsonAsync($"https://identitytoolkit.googleapis.com/v2/accounts/mfaEnrollment:start?key={config.ApiKey}", startEnrollmentRequest);
var startEnrollmentResponseBody = await startEnrollmentResponse.Content.ReadFromJsonAsync<TotpSessionInfoWrapper>();

var qrCode = startEnrollmentResponseBody.TotpSessionInfo.GetQRCodeUrl(userCredential.User.Info.Email, "studies-totp");
Console.WriteLine("OTP key generated, show this as an QR code");
// https://www.qrcode-monkey.com/
// TODO: Stop using other services and create an cli compatible QR Code generator.
Console.WriteLine(qrCode);

After scanning the QR Code on the OTP app, use the timed password generated to finalize MFA TOTP enrollment. You will recieve an email confirming this operation.

In [ ]:
var otp = "⚠️Sensitive data⚠️";
// TODO: Read this from console somehow? jupyter notebook doesnt seem to support it.

var finalizeEnrollmentRequest = new
{
    idToken = token,
    displayName = "⚠️Sensitive data⚠️",
    totpVerificationInfo = new { sessionInfo = startEnrollmentResponseBody.TotpSessionInfo.SessionInfo, VerificationCode = otp }
};

var finalizeEnrollmentResponse = await client.PostAsJsonAsync($"https://identitytoolkit.googleapis.com/v2/accounts/mfaEnrollment:finalize?key={config.ApiKey}", finalizeEnrollmentRequest);